# Welcome to my Capstone project !

In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests

In [3]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M"
raw=requests.get(url).text
#Get Text Version of LXML code

soup = BeautifulSoup(raw,"lxml")
My_table = soup.find("table",{"class":"wikitable sortable"}) #obtain only table code and its contents from the Wikipedia Page
pcode=[]
borou=[]
neigh=[]

for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells)==3:
        pcode.append(cells[0].find(text=True))
        borou.append(cells[1].find(text=True))
        neigh.append(cells[2].find(text=True))
        
column=['Postcode','Borough','Neighborhood']
mainframe=pd.DataFrame(columns=column) #Generate empty dataframe

for i in range(0,len(pcode)): #Add all data into dataframe, except 'Not assigned' Boroughs
    if borou[i]!="Not assigned":
        mainframe.loc[i]=[pcode[i],borou[i],neigh[i]]

mainframe.reset_index(drop=True)
pcodes=mainframe['Postcode'].unique()     #Get list of postal codes

In [6]:
for i in pcodes:
    indexes=mainframe.loc[mainframe['Postcode']==i].index #Get list of indexes corresponding to each postcode
    text=mainframe['Neighborhood'][indexes[0]].replace('\n','')
    for j in range(0,len(indexes)):
        if j !=0:
            text_add=mainframe['Neighborhood'][indexes[j]].replace('\n','')
            text=text+", "+text_add #Prepare new text data for each unique Postcode
            mainframe.drop(indexes[j],inplace=True) #Drop rows except the merged row
    mainframe['Neighborhood'][indexes[0]]=text

In [7]:
mainframe.reset_index(drop=True,inplace=True)

In [8]:
NAindexes=mainframe.loc[mainframe['Neighborhood']=='Not assigned'].index #Get list of indexes corresponding to 'Not assigned' Neighborhood
for a in NAindexes:
    mainframe['Neighborhood'][a]=mainframe['Borough'][a]

In [9]:
print("Number of shows in filtered dataframe: {}".format(mainframe.shape[0]))
mainframe.head(11)

Number of shows in filtered dataframe: 103


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"
